In [1]:
# !pip install transformers
!pip install transformers==2.8.0
!git clone https://github.com/sitingGZ/bert-sner.git && cd bert-sner
!pip install pytorch_lightning
%cd /content/bert-sner/modules

  Using cached transformers-2.8.0-py3-none-any.whl (563 kB)
  Using cached tokenizers-0.5.2.tar.gz (64 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached boto3-1.28.71-py3-none-any.whl (135 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached botocore-1.31.71-py3-none-any.whl (11.3 MB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.7.0-py3-none-any.whl (79 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which 

In [2]:
!pwd

/content/bert-sner/modules


In [12]:
# !ls ../../gdrive/MyDrive

In [3]:
import warnings

# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

warnings.filterwarnings('ignore')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import sys
# sys.path.append('modules')

import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, EncoderDecoderConfig
from BERT2span_semantic_disam import BERT2span
from helpers import load_config, set_seed
from inference import final_label_results_rescaled

base_name =  "bert-base-german-cased"
configs = load_config('../configs/step3_gpu_span_semantic_group.yaml')
tokenizer = AutoTokenizer.from_pretrained(base_name)
bertMLM = AutoModelForMaskedLM.from_pretrained(base_name)
bert_sner = BERT2span(configs, bertMLM, tokenizer)

checkpoint_path = "../../gdrive/MyDrive/ner-models/german_bert_ex4cds_500_semantic_term.ckpt"
# checkpoint_path = "../../gdrive/MyDrive/ner-models/german_bert_muchmore_semantic_term.ckpt"
state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
# bertMLM.load_state_dict(state_dict, strict=False)
bert_sner.load_state_dict(state_dict, strict=False)
bert_sner.eval()

suggested_terms = {'Condition': 'Zeichen oder Symptom',
               'DiagLab': 'Diagnostisch und Laborverfahren',
                'LabValues': 'Klinisches Attribut',
                 'HealthState': 'Gesunder Zustand',
                 'Measure': 'Quantitatives Konzept',
                 'Medication': 'Pharmakologische Substanz',
                 'Process': 'Physiologische Funktion',
                 'TimeInfo': 'Zeitliches Konzept'}

words = "Welche Erreger verursachen häufig eine frühe postoperative Osteitis oder periimplantäre Infektion?".split()
words_list = [words]
heatmaps, ner_results = final_label_results_rescaled(words_list, tokenizer, bert_sner, suggested_terms, threshold=0.5)


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:

ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 0, 'O'),
  ('verursachen', 5, 'Measure'),
  ('häufig', 0, 'O'),
  ('eine', 0, 'O'),
  ('frühe', 2, 'DiagLab'),
  ('postoperative', 0, 'O'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 3, 'LabValues'),
  ('Infektion?', 1, 'Condition')]]

In [9]:
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 3, 'LabValues'),
  ('verursachen', 5, 'Measure'),
  ('häufig', 5, 'Measure'),
  ('eine', 0, 'O'),
  ('frühe', 2, 'DiagLab'),
  ('postoperative', 6, 'Medication'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 3, 'LabValues'),
  ('Infektion?', 1, 'Condition')]]

In [21]:
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 0, 'O'),
  ('verursachen', 0, 'O'),
  ('häufig', 0, 'O'),
  ('eine', 0, 'O'),
  ('frühe', 8, 'TimeInfo'),
  ('postoperative', 0, 'O'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 7, 'Process'),
  ('Infektion?', 2, 'DiagLab')]]

In [19]:
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 0, 'O'),
  ('verursachen', 0, 'O'),
  ('häufig', 0, 'O'),
  ('eine', 0, 'O'),
  ('frühe', 8, 'TimeInfo'),
  ('postoperative', 4, 'HealthState'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 7, 'Process'),
  ('Infektion?', 2, 'DiagLab')]]